## Reference counting

In [1]:
import sys

### Reference counting

In [2]:
def foo():
    a = []
    b = a
    print(sys.getrefcount(a))

    del b
    print(sys.getrefcount(a))

foo()

3
2


### Reference Cycle

In [6]:
def foo():
    x = []
    x.append(x)
    print(sys.getrefcount(x))
    del x

3


### GC

### Mannual GC

In [8]:
!python3 gc.py

3
2
GC collected:  0


In [9]:
!python3 gc_ref_cycle.py

3
GC collected:  1


### Auto GC

#allocation - #de-allocation > threshold → run gc

In [9]:
import gc
gc.get_threshold() 

(700, 10, 10)

threshold = 70

## GIL

In [4]:
import sys

In [8]:
# get the interpreter’s thread switch interval (in seconds)
sys.getswitchinterval()

0.005

### Why need threading.Lock if we have the GIL?

In [10]:
import threading
import dis

#### Unsafe thread

In [182]:
%%time
class Counter:
    def __init__(self):
        self.count = 0

    def increase(self, n):
        for _ in range(n):
            self.count += 1

counter = Counter()
n_increase = 100000
n_threads = 5
threads = []
for i in range(n_threads):
    t = threading.Thread(target=counter.increase, args=(n_increase, ))
    threads.append(t)

for t in threads:
    t.start()

for t in threads:
    t.join()

print(counter.count)

488525
CPU times: user 30.7 ms, sys: 0 ns, total: 30.7 ms
Wall time: 31.1 ms


In [133]:
dis.dis(counter.increase)

  6           0 LOAD_GLOBAL              0 (range)
              2 LOAD_FAST                1 (n)
              4 CALL_FUNCTION            1
              6 GET_ITER
        >>    8 FOR_ITER                18 (to 28)
             10 STORE_FAST               2 (_)

  7          12 LOAD_FAST                0 (self)
             14 DUP_TOP
             16 LOAD_ATTR                1 (count)
             18 LOAD_CONST               1 (1)
             20 INPLACE_ADD
             22 ROT_TWO
             24 STORE_ATTR               1 (count)
             26 JUMP_ABSOLUTE            8
        >>   28 LOAD_CONST               0 (None)
             30 RETURN_VALUE


### Atomic operation
`sort()` is atomic

`+=` is not

In [17]:
def foo():
    a = 1
    a += 1

dis.dis(foo)

  2           0 LOAD_CONST               1 (1)
              2 STORE_FAST               0 (a)

  3           4 LOAD_FAST                0 (a)
              6 LOAD_CONST               1 (1)
              8 INPLACE_ADD
             10 STORE_FAST               0 (a)
             12 LOAD_CONST               0 (None)
             14 RETURN_VALUE


In [16]:
def foo():
    lst = [4, 1]
    lst.sort()

dis.dis(foo)

  2           0 LOAD_CONST               1 (4)
              2 LOAD_CONST               2 (1)
              4 BUILD_LIST               2
              6 STORE_FAST               0 (lst)

  3           8 LOAD_FAST                0 (lst)
             10 LOAD_METHOD              0 (sort)
             12 CALL_METHOD              0
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


#### Fix

In [179]:
%%time
class Counter2:
    def __init__(self):
        self.count = 0
        self.lock = threading.Lock()

    def increase(self, n):
        for _ in range(n):
            self.lock.acquire()
            self.count += 1
            self.lock.release()

counter2 = Counter2()
n_increase = 100000
n_threads = 5
threads = []
for i in range(n_threads):
    t = threading.Thread(target=counter2.increase, args=(n_increase, ))
    threads.append(t)

for t in threads:
    t.start()

for t in threads:
    t.join()

print(counter2.count)

500000
CPU times: user 243 ms, sys: 234 ms, total: 476 ms
Wall time: 287 ms


In [146]:
dis.dis(counter2.increase)

  7           0 LOAD_GLOBAL              0 (range)
              2 LOAD_FAST                1 (n)
              4 CALL_FUNCTION            1
              6 GET_ITER
        >>    8 FOR_ITER                38 (to 48)
             10 STORE_FAST               2 (_)

  8          12 LOAD_FAST                0 (self)
             14 LOAD_ATTR                1 (lock)
             16 LOAD_METHOD              2 (acquire)
             18 CALL_METHOD              0
             20 POP_TOP

  9          22 LOAD_FAST                0 (self)
             24 DUP_TOP
             26 LOAD_ATTR                3 (count)
             28 LOAD_CONST               1 (1)
             30 INPLACE_ADD
             32 ROT_TWO
             34 STORE_ATTR               3 (count)

 10          36 LOAD_FAST                0 (self)
             38 LOAD_ATTR                1 (lock)
             40 LOAD_METHOD              4 (release)
             42 CALL_METHOD              0
             44 POP_TOP
             46 JU